In [9]:
import tweepy
import pandas as pd
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [10]:
%run ./settings.py

In [11]:
def scrape_tweet(word, timeline, tweet_count):
    df = pd.DataFrame(columns=['username', 'location', 'following',
                               'followers', 'retweetcount', 'text', 'hashtags', 'time'])
    tweets = tweepy.Cursor(api.search, q=word, lang="en",
                           since=timeline, tweet_mode='extended').items(tweet_count)
    list_tweets = [tweet for tweet in tweets]
    i = 1  
    for tweet in list_tweets:
        username = tweet.user.screen_name
        location = tweet.user.location
        following = tweet.user.friends_count
        followers = tweet.user.followers_count
        time = tweet.created_at
        retweetcount = tweet.retweet_count
        hashtags = tweet.entities['hashtags']
        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:
            text = tweet.full_text
        hashtext = list()
        for j in range(0, len(hashtags)):
            hashtext.append(hashtags[j]['text'])
        tweet = [username, location, following,
                     followers, retweetcount, text, hashtext, time]
        df.loc[len(df)] = tweet
        i = i+1

    return df

In [12]:
def sentiment_analyser(df):
    sid = SentimentIntensityAnalyzer()
    df['scores'] = df['text'].apply(lambda text: sid.polarity_scores(text))
    df['compound']  = df['scores'].apply(lambda score_dict: score_dict['compound'])
    df['Sentiment'] = df['compound'].apply(lambda c: 'Positive' if c >=0 else 'Negative')
    return df

In [14]:
def sentiment_analyser_metrics(df):
    pos = df['Sentiment'].value_counts()[0] / len(df['Sentiment'])
    neg = df['Sentiment'].value_counts()[1] / len(df['Sentiment'])
    print('Percent of positive sentiment:', pos*100,'%')
    print('Percent of negative sentiment:', neg*100,'%')

In [15]:
if __name__ == '__main__':
      
    # Enter your own credentials obtained 
    # from your developer account
    consumer_key = API_KEY
    consumer_secret = API_KEY_SECRET
    access_key = ACCESS_TOKEN
    access_secret = ACCESS_TOKEN_SECRET
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
      
    # Enter Hashtag and initial date
    print("Enter Twitter HashTag to search for (without #)")
    word = input()
    print("Enter Date since The Tweets are required in yyyy-mm--dd")
    timeline = input()
    print("Enter the number of tweets to be retrieved for the given timeline (numbers only)")
    tweet_count = int(input())
      
    # number of tweets you want to extract in one run
    scrape_tweet(word, timeline, tweet_count)
    df = scrape_tweet(word, timeline, tweet_count)
    df = sentiment_analyser(df)
    sentiment_analyser_metrics(df)

Enter Twitter HashTag to search for
Enter Date since The Tweets are required in yyyy-mm--dd
Percent of positive sentiment: 95.0 %
Percent of negative sentiment: 5.0 %
Total Tweets for the selected time range: 100
